# Personalised Federated Learning

This is a notebook for the Principles of ML Systems project. Just a scrap pad, don't read too much into it!



## Set Up Tensorboard

In [8]:
from tensorboard import notebook


%load_ext tensorboard 

%tensorboard --logdir=./runs --bind_all --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from glob import glob
import os
import numpy as np 

for directory,_ , _ in os.walk("./completed_runs"):
    clients_dirs = glob(directory + "/client*/")


    client_accuracies = []
    all_cl_accs = []

    for client_d in clients_dirs:
        event_acc = EventAccumulator(client_d)
        event_acc.Reload()
        # Show all tags in the log file
        # print(event_acc.Tags())
        # E. g. get wall clock, number of steps and value for a scalar 'Accuracy'|
        #print(*event_acc.Scalars('Accuracy/test'))
        _, steps, test_acc = zip(*event_acc.Scalars('Accuracy/test'))
        client_accuracies.append(test_acc[-1])
        all_cl_accs.append(zip(steps,test_acc))

    if client_accuracies:
        print("")
        print(directory)
        #print([ f"{a} {x}" for a,x in all_cl_accs[2]])
        mean = np.mean(client_accuracies)
        std = np.std(client_accuracies)
        x = np.array(client_accuracies)
        idx = (x).argsort()[-2:]
        top2 = np.mean(x[idx])
        idx = (x).argsort()[:2]
        bottom2 = np.mean(x[idx])
         
        print(f'{mean:.3f} & {std:.3f} & {top2 :.3f} & {bottom2 :.3f}')


./completed_runs/cifar10/qllr=1 q=0.01 alpha=0.01
0.509 & 0.097 & 0.623 & 0.405

./completed_runs/cifar10/qffl  = 1e-2 - same as alpha
0.100 & 0.090 & 0.200 & 0.010

./completed_runs/cifar10/personalisation/before_personalisation/qfed/q=0.1/before_personalisation
0.189 & 0.035 & 0.234 & 0.145

./completed_runs/cifar10/personalisation/before_personalisation/qfed/q=0.01/before_personalisation
0.387 & 0.072 & 0.464 & 0.308

./completed_runs/cifar10/personalisation/before_personalisation/fedavg/round2
0.482 & 0.063 & 0.562 & 0.407

./completed_runs/cifar10/personalisation/before_personalisation/fedavg/round3
0.491 & 0.044 & 0.553 & 0.431

./completed_runs/cifar10/personalisation/before_personalisation/fedavg/round1
0.487 & 0.168 & 0.664 & 0.310

./completed_runs/cifar10/personalisation/after_personalisation/qfed/q=0.1
0.151 & 0.137 & 0.301 & 0.012

./completed_runs/cifar10/personalisation/after_personalisation/qfed/q=0.01
0.333 & 0.042 & 0.385 & 0.281

./completed_runs/cifar10/personalisa

In [10]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from glob import glob
import os
import numpy as np 
import matplotlib.pyplot as plt

def set_size(fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    width = 397.48499
    
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim
    
# Using seaborn's style
plt.style.use('seaborn')

# With LaTex fonts
tex_fonts = {
# Use LaTeX to write all text
"text.usetex": False,
"font.family": "serif",
# Use 10pt font in plots, to match 10pt font in document
"axes.labelsize": 10,
"font.size": 10,
# Make the legend/label fonts a little smaller
"legend.fontsize": 8,
"xtick.labelsize": 8,
"ytick.labelsize": 8
}
plt.rcParams.update(tex_fonts)

for directory,_ , _ in os.walk("./completed_runs"):
    clients_dirs = glob(directory + "/client*/")

    

    
    # Initialise figure instance
    test_fig, test_ax = plt.subplots(1, 1, figsize=set_size(0.45))
    train_fig, train_ax = plt.subplots(1, 1, figsize=set_size(0.45))

    # Plot
    test_ax.set_xlim(0, 50)
    test_ax.set_xlabel("Epochs")
    test_ax.set_ylabel("Accuracy (test)")
    
    train_ax.set_xlim(0, 50)
    train_ax.set_xlabel("Epochs")
    train_ax.set_ylabel("Accuracy (train)")
    
    test_accs = []
    train_accs = []

    for client_d in clients_dirs:
        event_acc = EventAccumulator(client_d)
        event_acc.Reload()
        # Show all tags in the log file
        # E. g. get wall clock, number of steps and value for a scalar 'Accuracy'|
        #print(*event_acc.Scalars('Accuracy/test'))
        #_, test_step, test_acc = zip(*event_acc.Scalars('Accuracy/test'))
        #test_accs.append((test_step,test_acc))
        _, train_step, train_acc = zip(*event_acc.Scalars('Accuracy/train'))
        train_accs.append((train_step,train_acc))
        
        
    if test_accs:
        for (x,y) in test_accs:
            test_ax.plot(x, y)
        test_fig.savefig(directory + '/test_acc.pdf', format='pdf', bbox_inches='tight')
    if train_accs:
        for (x,y) in train_accs:
            print(directory)
            train_ax.plot(x, y)
        train_fig.savefig(directory + '/train_acc.pdf', format='pdf', bbox_inches='tight')
    plt.close(train_fig)
    plt.close(test_fig)


./completed_runs/cifar10/qllr=1 q=0.01 alpha=0.01
./completed_runs/cifar10/qffl  = 1e-2 - same as alpha
./completed_runs/cifar10/personalisation/before_personalisation/qfed/q=0.1/before_personalisation
./completed_runs/cifar10/personalisation/before_personalisation/qfed/q=0.01/before_personalisation
./completed_runs/cifar10/personalisation/before_personalisation/fedavg/round2
./completed_runs/cifar10/personalisation/before_personalisation/fedavg/round3
./completed_runs/cifar10/personalisation/before_personalisation/fedavg/round1
./completed_runs/cifar10/personalisation/after_personalisation/qfed/q=0.1
./completed_runs/cifar10/personalisation/after_personalisation/qfed/q=0.01
./completed_runs/cifar10/personalisation/after_personalisation/fedavg/fed trial round
./completed_runs/cifar10/personalisation/after_personalisation/fedavg/fedavg alpha 0.01
./completed_runs/cifar10/personalisation/after_personalisation/fedavg/alpha 0.1
./completed_runs/cifar10/qfflr = 0.1 q = 0.05
./completed_runs